# PySLAMMER comparison with analytical

In [ ]:
import csv
import numpy as np
from matplotlib import pyplot as plt
from pyslammer.rigid_analysis import RigidAnalysis

G_EARTH = 9.80665

# Open the harmonic solutions file and read the data. User to modify the file path as needed.
file = open('D:/Users/donal/Documents/Python/SlidingBlock/Records/harmonic_solutions.csv')
if file is None:
    exit()
reader = csv.reader(file)
freq = []
k_y = []
analytical_displacement = []
for row in reader:
    if '#' in row[0]:
        continue
    freq.append(float(row[0]))
    k_y.append(float(row[1]))
    analytical_displacement.append(float(row[2]))
file.close()

# Calculate the displacement for each frequency and ky value.
disp = []
error = []
for i in range(len(freq)):
    # Create time and acceleration arrays using each frequency and ky value
    # for one half of the period.
    dt = 0.001
    time = np.arange(0, 1/(freq[i]), dt)
    accel = np.sin(freq[i]*2*np.pi*time)
    time_history = np.vstack((time, accel))

    # Calculate the displacement using the desired method and append the total
    # displacement to the disp list. User to modify the next three lines as
    # needed to test the desired method.
    data = RigidAnalysis(accel, dt, k_y[i])
    disp.append(data.block_disp[-1])

    # Calculate the percent error between the analytical solution and the calculated.
    error.append(abs(analytical_displacement[i] - data.block_disp[-1]) * 100)

# Create a data array to hold frequency and error data (dimension 1) sorted by frequency 
# tested (dimension 1) and ky value (dimension 2).
data = np.ndarray(shape=(2, 11, 6))
ky_analyzed = [0.1, 0.2, 0.4, 0.6, 0.8, 1]
ky_idx = [0] * len(ky_analyzed)

i = 0
for i in range(len(freq)):
    # Find the index of the ky value in the ky_analyzed list to save frequency and error 
    # data in the correct location in the data array based on ky.
    idx = ky_analyzed.index(float(k_y[i]))
    data[0][ky_idx[idx]][idx] = float(freq[i])
    data[1][ky_idx[idx]][idx] = float(error[i])
    ky_idx[idx] += 1
    i += 1

# Plot the error vs frequency data for each ky value.
fig, ax = plt.subplots()
for j in range(len(ky_analyzed)):
    ax.plot(data[0, :, j], data[1, :, j], label='ky = ' + str(ky_analyzed[j]))
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Error (%)')
ax.set_xscale('log')
ax.legend()
plt.show()